# Using `tesswcs` to find observable targets

`tesswcs` has some convenience functions to help you find out which targets are observable with TESS. The `get_pixel_locations` functon will accept sky coordinates, and return a table of which sector, camera, CCD, row, and column those targets fall on (if any). 

Let's take a look at how to use it. 

## Example 1: Finding where transiting planets fall on TESS pixels

In [1]:
from tesswcs.locate import get_pixel_locations
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.table import Table

First, we need a target list to check the observability of. For this purpose, I will use the list of confirmed exoplanets. Below I use astroquery to get the coordinates of all the confirmed, transiting exoplanets from the [Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html).

In [2]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
confirmed_exoplanets_table = NasaExoplanetArchive.query_criteria(table="ps", select="pl_name,ra,dec,sy_tmag", where="default_flag=1 AND tran_flag=1")
confirmed_exoplanets_table.sort('sy_tmag')

In [3]:
confirmed_exoplanets_table

pl_name,ra,dec,sy_tmag,sky_coord
,deg,deg,mag,"deg,deg"
str19,float64,float64,float64,SkyCoord
HD 219134 b,348.3372026,57.1696255,4.6278,"348.3372026,57.1696255"
HD 219134 c,348.3372026,57.1696255,4.6278,"348.3372026,57.1696255"
HR 810 b,40.6417178,-50.7993484,4.8754,"40.6417178,-50.7993484"
HD 136352 b,230.4401147,-48.3188174,5.0494,"230.4401147,-48.3188174"
HD 136352 c,230.4401147,-48.3188174,5.0494,"230.4401147,-48.3188174"
HD 136352 d,230.4401147,-48.3188174,5.0494,"230.4401147,-48.3188174"
...,...,...,...,...
POTS-1 b,203.6087259,-66.5811639,16.2949,"203.6087259,-66.5811639"


In [4]:
confirmed_exoplanets_table['sky_coord']

<SkyCoord (ICRS): (ra, dec) in deg
    [(348.3372026,  57.1696255), (348.3372026,  57.1696255),
     ( 40.6417178, -50.7993484), ..., (269.1479583, -29.5392222),
     (269.7246667, -29.1890556), (269.7583333, -29.1981944)]>

I can now pass these `SkyCoord`s to `get_pixel_location` to get a table of the regions those targets fall on. Any targets that do not fall on a pixel will be omitted.

In [5]:
observable = get_pixel_locations(confirmed_exoplanets_table['sky_coord'])

We can use pandas to learn more about this table. For example, how many planets are observable for the first time during cycle 7?

In [6]:
g = observable.to_pandas().groupby('Target Index').min()
g

,Sector,Camera,CCD,Row,Column
Target Index,,,,,
0,17,1,1,106.168981,108.588031
1,17,1,1,106.168981,108.588031
2,2,3,1,496.345263,99.519309
3,12,1,1,108.697144,204.974722
4,12,1,1,108.697144,204.974722
...,...,...,...,...,...
4374,14,2,1,1.462736,14.583116
4375,12,1,1,1115.376413,148.932139
4376,39,1,2,28.095010,357.586512


This groupby operation takes the `min` across all the columns for cases of the same `'Target Index'`

In [11]:
cycle7_mask = (g['Sector'] >= 84) & (g['Sector'] <= 96)
target_index = list(g[cycle7_mask].index)
print(len(target_index))

113


There are 113 confirmed transiting planets that will be observed for the first time in Cycle 7!

In [12]:
confirmed_exoplanets_table[target_index]

pl_name,ra,dec,sy_tmag,sky_coord
,deg,deg,mag,"deg,deg"
str19,float64,float64,float64,SkyCoord
HD 106315 b,183.4724742,-0.3934357,8.56338,"183.4724742,-0.3934357"
HD 106315 c,183.4724742,-0.3934357,8.56338,"183.4724742,-0.3934357"
HD 164604 b,270.7787177,-28.5608343,8.6834,"270.7787177,-28.5608343"
Wolf 503 b,206.8461979,-6.1393369,9.2493,"206.8461979,-6.1393369"
HD 137496 b,231.7420543,-16.5090014,9.29961,"231.7420543,-16.5090014"
K2-292 b,205.3758786,-9.9460704,9.31975,"205.3758786,-9.9460704"
...,...,...,...,...
K2-52 b,246.6114157,-24.9699462,14.2904,"246.6114157,-24.9699462"


## Example 2: Finding whether a transient will be observable with TESS

Sometimes we might find an interesting transient, and we want to know whether it will fall on a TESS pixel so that we can recover the optical time-series when the data is downlinked. For example, [GRB 231106A](https://gcn.nasa.gov/circulars/34956) occured in November 2023 and happened to be observable during the TESS survey. 

Here we show how we would find out if a transient has been (or will be) observable with TESS. 

To do this we will need two things, the coordinate of the object, and the time at which we would like to observe it. Here we'll use the location and detection time of GRB 231106A.

In [13]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

t = Time('2023-11-06T18:13:23Z')
c = SkyCoord(113.4482, 29.2245, unit='degree')

In [14]:
get_pixel_locations(coords=c, time=t)

Target Index,Sector,Camera,CCD,Row,Column
int64,int64,int64,int64,float64,float64
0,71,4,1,1452.4469788301963,765.6934015886932


We can see that this returned a hit, we are able to detect the target during Sector 71, on Camera 4, CCD 1! 

In instances where the time passed is recent (or in the future!) and there is no archival TESS WCS to use, `tesswcs` will predict the WCS for future TESS pointings.